In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs
from bs4.element import Comment
import codecs
import collections
import pandas as pd
import numpy as np
import scipy as scp
import re
import os
from unicodedata import normalize

In [94]:
# utizado para baixar as páginas html que foram utilizadas no projeto

padrao = '((www\.)?[a-zA-Z0-9]+\.[a-zA-Z0-9]+(\.[a-zA-Z0-9]+)*)'
for i, site in enumerate(data_set.url):
    name = str(i) + re.findall(padrao, site)[0][0] + ".html"
    getPage(url=site, name="sites/" + name, codec="utf-8")
    print "finalizou " + name

In [2]:
# troca os carecteres com acento por caracteres sem acento e retorna apenas os caracteres alphanumericos

def remover_acentos_e_pontuacao(txt):
    sem_acentos = normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')
    return re.sub('[^a-zA-Z0-9 ]', '', sem_acentos)


In [5]:
# função utilizada para baixar uma página html e salvar em name
def getPage(url, name, codec):
    
    if not os.path.isfile(name):
        page = rq.get(url)
        f = codecs.open(name, 'w', codec)
        f.write(page.text)
        f.close

In [36]:
# utilizado para filtrar apenas of elementos html que são visiveis para o usuário
def tag_visible(element):
    
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

# utilizado para pegar o titulo da página
def title_page(element):
    
    if element.parent.name in ['title']:
        return True
    return False

# utilizado para pegar apenas o texto da página html
def getText(arquivo):        
    bsFile = bs(open(arquivo),"html.parser")
    texts = bsFile.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    title = filter(title_page, texts)
    words = [t.strip() for t in visible_texts] # pega as palavras da página
    words_title = [("title" + str(t.strip())) for t in title] # pega o titulo da página
    words = filter(lambda x: len(x)>0, words) # filtra apenas as palavras que tem pelo menos 1 caracter
    words_title = filter(lambda x: len(x)>0, words_title)
    words = list(map(lambda x: x.lower(), words)) # coloca todas as palavras em minusculo para evitar diferenciação de maiusculas e minusculas
    words_title = list(map(lambda x: x.lower(), words_title))
    words += words_title 
    text = " ".join(words)
    return remover_acentos_e_pontuacao(text)

# utilizado para pegar as palavras que aparecem em uma página html (apenas páginas salvas localmente)
def getUniqueWords(arquivo):
    text = getText(arquivo)
    words = text.split(" ") # transforma o texto em lista de palavras
    words = filter(lambda word: len(word) > 0, words) # filtrar por palavras não vazias
    uniqueWords = collections.Counter(words) # conta quantas ocorencias tem de cada palavra
    return uniqueWords

In [51]:
# pega as palavras unicas dos sites
fr = codecs.open("palavrasAcentoTudo.txt", "w", "utf-8") # seleciona o arquivo que vai salvar as palavras
files = os.listdir("sites")
# files = filter(lambda x: x[-5:].lower() == ".html", files) 
words = []
for index, f in enumerate(files):
    w = getUniqueWords("sites/" + f)
    words += list(w.keys())
    print (str(index) + " está no arquivo: " + str(f))
    
uwords = []
for w in words:
    if w not in uwords:
        uwords.append(w)
        
fr.write(",".join(uwords) + "\n")
fr.close()
    

0 está no arquivo: 31www.lojasrenner.com.br.html
1 está no arquivo: 148www.gregory.com.br.html
2 está no arquivo: 142www.gregory.com.br.html
3 está no arquivo: 137www.gregory.com.br.html
4 está no arquivo: 209www.dafiti.com.br.html
5 está no arquivo: 164www.zattini.com.br.html
6 está no arquivo: 27www.lojasrenner.com.br.html
7 está no arquivo: 155www.zattini.com.br.html
8 está no arquivo: 116www.cea.com.br.html
9 está no arquivo: 90www.marisa.com.br.html
10 está no arquivo: 102www.marisa.com.br.html
11 está no arquivo: 67mariadonata.com.br.html
12 está no arquivo: 42www.damyller.com.br.html
13 está no arquivo: 194produto.mercadolivre.com.br.html
14 está no arquivo: 201www.dafiti.com.br.html
15 está no arquivo: 208dafitimag.dafiti.com.br.html
16 está no arquivo: 169www.zattini.com.br.html
17 está no arquivo: 51www.damyller.com.br.html
18 está no arquivo: 145www.gregory.com.br.html
19 está no arquivo: 130www.gregory.com.br.html
20 está no arquivo: 126www.cea.com.br.html
21 está no arquiv

172 está no arquivo: 190produto.mercadolivre.com.br.html
173 está no arquivo: 129www.cea.com.br.html
174 está no arquivo: 45blog.damyller.com.br.html
175 está no arquivo: 33www.lojasrenner.com.br.html
176 está no arquivo: 95www.marisa.com.br.html
177 está no arquivo: 107www.marisa.com.br.html
178 está no arquivo: 76mariadonata.com.br.html
179 está no arquivo: 220www.dafiti.com.br.html
180 está no arquivo: 212www.dafiti.com.br.html
181 está no arquivo: 109www.cea.com.br.html
182 está no arquivo: 40www.lojasrenner.com.br.html
183 está no arquivo: 195produto.mercadolivre.com.br.html
184 está no arquivo: 81mariadonata.com.br.html
185 está no arquivo: 62www.damyller.com.br.html
186 está no arquivo: 86pesquisa.marisa.com.br.html
187 está no arquivo: 172www.zattini.com.br.html
188 está no arquivo: 204www.dafiti.com.br.html
189 está no arquivo: 114www.cea.com.br.html
190 está no arquivo: 119www.cea.com.br.html
191 está no arquivo: 71mariadonata.com.br.html
192 está no arquivo: 53www.damyller.c

In [18]:
fr = codecs.open("palavrasAcentoSelecionadas.txt", "r", "utf-8")
uwordsSelected = fr.read().split(",")
fr.close()

In [43]:
# utilizado para ler o arquivo com as páginas utilizadas no projeto
data_set = pd.read_csv("sites.csv")

In [44]:
# utilizado para pegar as repetições de cada palavra em cada página (Base de treinamento)
fr = codecs.open("baseSelecionados.arff", "w", "utf-8")

fr.write("@RELATION iris\n\n")
for u in uwordsSelected:
    fr.write("@ATTRIBUTE " + u + " NUMERIC\n")

fr.write("@ATTRIBUTE classePagina {não, sim}\n")
fr.write("\n\n@DATA\n")


files = os.listdir("sites")
# files = filter(lambda x: x[-5:].lower() == ".html", files)
for index, f in enumerate(files):
    allWords = getUniqueWords("sites" + "/" + f)

    inputSite = []
    for pp in uwordsSelected:
        inputSite.append(allWords[pp])
    
    inputSite = ",".join(map(str, inputSite))
    
    padrao = '([0-9]+)'
    fr.write(inputSite + "," + data_set.anuncio[int(re.findall(padrao, f)[0])] + "\n")
    print (str(index) + ", está no arquivo: " + str(f))

fr.close()

0, está no arquivo: 31www.lojasrenner.com.br.html
1, está no arquivo: 148www.gregory.com.br.html
2, está no arquivo: 142www.gregory.com.br.html
3, está no arquivo: 137www.gregory.com.br.html
4, está no arquivo: 209www.dafiti.com.br.html
5, está no arquivo: 164www.zattini.com.br.html
6, está no arquivo: 27www.lojasrenner.com.br.html
7, está no arquivo: 155www.zattini.com.br.html
8, está no arquivo: 116www.cea.com.br.html
9, está no arquivo: 90www.marisa.com.br.html
10, está no arquivo: 102www.marisa.com.br.html
11, está no arquivo: 67mariadonata.com.br.html
12, está no arquivo: 42www.damyller.com.br.html
13, está no arquivo: 194produto.mercadolivre.com.br.html
14, está no arquivo: 201www.dafiti.com.br.html
15, está no arquivo: 208dafitimag.dafiti.com.br.html
16, está no arquivo: 169www.zattini.com.br.html
17, está no arquivo: 51www.damyller.com.br.html
18, está no arquivo: 145www.gregory.com.br.html
19, está no arquivo: 130www.gregory.com.br.html
20, está no arquivo: 126www.cea.com.br.h

168, está no arquivo: 25www.lojasrenner.com.br.html
169, está no arquivo: 29www.lojasrenner.com.br.html
170, está no arquivo: 124www.cea.com.br.html
171, está no arquivo: 175www.zattini.com.br.html
172, está no arquivo: 190produto.mercadolivre.com.br.html
173, está no arquivo: 129www.cea.com.br.html
174, está no arquivo: 45blog.damyller.com.br.html
175, está no arquivo: 33www.lojasrenner.com.br.html
176, está no arquivo: 95www.marisa.com.br.html
177, está no arquivo: 107www.marisa.com.br.html
178, está no arquivo: 76mariadonata.com.br.html
179, está no arquivo: 220www.dafiti.com.br.html
180, está no arquivo: 212www.dafiti.com.br.html
181, está no arquivo: 109www.cea.com.br.html
182, está no arquivo: 40www.lojasrenner.com.br.html
183, está no arquivo: 195produto.mercadolivre.com.br.html
184, está no arquivo: 81mariadonata.com.br.html
185, está no arquivo: 62www.damyller.com.br.html
186, está no arquivo: 86pesquisa.marisa.com.br.html
187, está no arquivo: 172www.zattini.com.br.html
188, 

In [41]:
# utilizado para pegar as repetições de cada palavra em cada página (predict)
fr = codecs.open("bruno.arff", "w", "utf-8")
files = os.listdir("html")

fr.write("@RELATION iris\n\n")
for u in uwordsSelected:
    fr.write("@ATTRIBUTE " + u + " NUMERIC\n")

fr.write("@ATTRIBUTE classePagina {não, sim}\n")
fr.write("\n\n@DATA\n")

# files = filter(lambda x: x[-5:].lower() == ".html", files)
for index, f in enumerate(files):
    allWords = getUniqueWords("html" + "/" + f)

    inputSite = []
    for pp in uwordsSelected:
        inputSite.append(allWords[pp])
    
    inputSite = ",".join(map(str, inputSite))
    
    padrao = '([0-9]+)'
    fr.write(inputSite + ",?\n")
    print (str(index) + ", está no arquivo: " + str(f))

fr.close()

0, está no arquivo: Sacola | Marisa
1, está no arquivo: Cardigans e Suéteres | Promoção de cardigans e suéteres na Marisa
2, está no arquivo: Marisa
3, está no arquivo: Dakota | Promoção de dakota na Marisa
4, está no arquivo: espelhado | Marisa
5, está no arquivo: Calça Feminina | Promoção de calça feminina na Marisa
6, está no arquivo: Login | Marisa
7, está no arquivo: DelRio | Promoção de delrio na Marisa
8, está no arquivo: Frozen | Promoção de frozen na Marisa
9, está no arquivo: Kit Dia das Mães - Vestido | Promoção de kit dia das mães - vestido na Marisa
10, está no arquivo: Marvel | Promoção de marvel na Marisa
11, está no arquivo: Bijuterias | Promoção de bijuterias na Marisa
12, está no arquivo: Camisa Feminina | Promoção de camisa feminina na Marisa
13, está no arquivo: Dia das Mães - Oferta | Promoção de dia das mães - oferta na Marisa
14, está no arquivo: Havaianas | Promoção de havaianas na Marisa
15, está no arquivo: Kit Dia Das Mães – Blusa Plus | Promoção de kit dia d